# Data Acquisition

## 1.0 Introduction
The project team are scrapping data from the following websites:
1. The Numbers: https://www.the-numbers.com/
2. The Academy Award Database: http://awardsdatabase.oscars.org/

The first half of the notebook contains the procedures taken to scrap the information from The Numbers and data wrangling which includes cleaning, feature engineering, and formating. From this website, the project team obtain the top 100 movies (Overall Revenue) from the past 10 years (aka from 2010 to 2019). 

The second half of the notebook contains the The first half of the notebook contains the procedures taken to scrap the information from The Academy Award Database and data wrangling which includes cleaning, feature engineering, and formating. From the Award Database, the project obtained all the nominatees from the Best Picture, Animated Feature Film, and Directing from 2010 to 2019. 

In [3]:
import requests
from bs4 import BeautifulSoup
from requests import get
import re
import pandas as pd

## 2.0 Data Acquisition from The Numbers

### 2.1 Building functions to exactract information
The following functions are built for making the for loops simpler when obtaining data

In [4]:
USER_AGENT = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36"
# US english
LANGUAGE = "en-US,en;q=0.5"

def get_soup(link):
    """Constructs and returns a soup using the HTML content of `url` passed"""
    # initialize a session
    session = requests.Session()
    # set the User-Agent as a regular browser
    session.headers['User-Agent'] = USER_AGENT
    # request for english content (optional)
    session.headers['Accept-Language'] = LANGUAGE
    session.headers['Content-Language'] = LANGUAGE
    # make the request
    html = session.get(url)
    # return the soup
    return bs(html.content, "html.parser")

In [5]:
def get_all_tables(soup):
    """Extracts and returns all tables in a soup object"""
    return soup.find_all("table")

In [6]:
def get_table_headers(table):
    """Given a table soup, returns all the headers"""
    headers = []
    for th in table.find("tr").find_all("th"):
        headers.append(th.text.strip())
    return headers

In [7]:
def get_table_rows(table):
    """Given a table, returns all its rows"""
    rows = []
    for tr in table.find_all("tr")[1:]:
        cells = []
        # grab all td tags in this table row
        tds = tr.find_all("td")
        if len(tds) == 0:
            # if no td tags, search for th tags
            # can be found especially in wikipedia tables below the table
            ths = tr.find_all("th")
            for th in ths:
                cells.append(th.text.strip())
        else:
            # use regular td tags
            for td in tds:
                cells.append(td.text.strip())
        rows.append(cells)
    return rows

In [8]:
def save_as_csv(table_name, headers, rows):
    pd.DataFrame(rows, columns=headers).to_csv(f"{table_name}.csv")

In [9]:
def main(url):
    print(url)
# get the soup
    soup = get_soup(url)
# extract all the tables from the web page
    tables = get_all_tables(soup)
    print(f"[+] Found a total of {len(tables)} tables.")
    for table in tables:
        headers = get_table_headers(table)
        rows = get_table_rows(table)
        table_name = f"table-{num}"
        print(f"[+] Saving {table_name}")
        save_as_csv(table_name, headers, rows)

In [ ]:
num = 0
url = 'https://www.the-numbers.com/movies/report/All/All/All/All/All/All/All/All/All/None/None/2020/2020/None/None/None/None/None/None?show-release-date=On&view-order-by=domestic-box-office&show-release-year=On&view-order-direction=desc&show-production-budget=On&show-opening-weekend-theaters=On&show-domestic-box-office=On&show-maximum-theaters=On&show-theatrical-engagements=On&show-international-box-office=On&show-opening-weekend-revenue=On&show-worldwide-box-office=On&show-worldwide-release-date=On&show-theatrical-distributor=On&show-source=On&show-creative-type=On'
main(url)

# iterate over the-numbers (based on the url characteristics)
for i in range(101,6000,+100):
    url = ('https://www.the-numbers.com/movie/budgets/all/' + str(i))
    main(url)
    num += 1

### 2.2. Combine all the data together

In [ ]:
# stitch together dataframes where axis = 0
import pandas as pd

def merge_f(x,y):
    df = pd.concat([x,y], axis = 0) # generalzing function so i can feed it my list of files
    return df

df = pd.read_csv('path/file_name_1.csv')

# iterate it over a range
for i in range(1,11): # represents the number of tables scraped from tn
    f = 'path/file_name_' + str(i) + '.csv' 
    print(f)
    y = pd.read_csv(f) # read and store the csv in a dummy variable
    df = merge_f(df,y) # name the merged and remerged df

df.shape # checking to make sure all the entries made it in

In [ ]:
df.to_csv(r'file_name.csv', index = False)

### 2.3 Data Cleaning 
As data has been combine together, the step involves identifying potential issues within the dataframe, adding new features, and make sure the format of values are consistent.  

In [ ]:
# Reading the raw data in
df_x = pd.read_csv('master_df.csv')

# Create a copy from the raw data into a new dataframe
df = df_x.copy()

# Drop a blank column
df.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
# Rename the columns to make it clear
df.rename(columns = {'Released': 'Year', 'Theatrical Distributor': 'TheatricalDistributor', 
                     'Production Method': 'ProductionMethod', 'Creative Type': 'CreativeType', 
                     'Domestic': 'DomesticGross', 'International': 'InternationalGross', 
                     'Worldwide': 'WorldwideGross'}, inplace = True)

In [ ]:
# Formatting the numbers to into integers and get rid of the $
df.Budget = df.Budget.str.replace('$','').str.replace('\,','').astype(int)
df.DomesticGross = df.DomesticGross.str.replace('$','').str.replace('\,','').astype(int)
df.InternationalGross= df.InternationalGross.str.replace('$','').str.replace('\,','').astype(int)
df.WorldwideGross = df.WorldwideGross.str.replace('$','').str.replace('\,','').astype(int)


In [ ]:
# Create New feature, net profit
df['NetProfits'] = df.WorldwideGross - df.Budget

In [ ]:
# Overwrite the dataframe into the csv file
df.to_csv(r'master_df.csv', index = False)

# 3.0 Data Acquisition from the Academy Awards
This is a searchable database, if directly using request method to obtain the html content, there is no information. An alternative method was used. The result page's html content was saved and uploaded onto github. The follow process will be using the html information that is in the project repo

## 3.1 Data Extraction

In [11]:
# Read in the raw html file, get the content and build a soup
url = 'https://raw.githubusercontent.com/xinegan88/some_files/master/results%20(1).html'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
keywords = soup.find_all('a', class_='nominations-link')

In [12]:
# Make a list and wrap in all the nomination information
list_1 = []
for tag in keywords:
    list_1.append(tag.text.strip())

### 3.2 Data Cleaning and Formatting

In [14]:
# The list contains nominations from the past 10 years
# Let's build a function to find where are the repeating keywords:
# The keywords: BEST PICTURE, DIRECTING, and ANIMATED FEATURE FILM
def find_dup(list_name, element):
    return [i for i, x in enumerate(list_name) if x == element]


# Using the function to find the index of those keywords
directing = find_dup(list_1, 'DIRECTING')
ani = find_dup(list_1, 'ANIMATED FEATURE FILM')
pic = find_dup(list_1, 'BEST PICTURE')

### 3.2.1 DIRECTING records

In [15]:
# Now, we know where the keyword (aka. the award category is)
# Extracting director information from the list
directors = []
for index in directing:
    directors.append(list_1[index:index+11])
    
# It is simple as every year as there are five nominations per year
x = []
y = []

for i in range(len(directors)):
    a = [directors[i][1], directors[i][3], 
                              directors[i][5], directors[i][7], directors[i][9]]
    b = [directors[i][2], directors[i][4], 
                              directors[i][6], directors[i][8], directors[i][10]]
    x.extend(a)
    y.extend(b)

In [16]:
# Build the directing nomination DataFrame
s = pd.Series(range(2010,2020)) # Range of years
# Repeat the series for 5 times as there are five nominees every year
df_directors = pd.DataFrame({'Year':s.repeat(5),'Movie':x,'Director':y})

df_directors.head()

,Year,Movie,Director
0,2010,Black Swan,Darren Aronofsky
0,2010,The Fighter,David O. Russell
0,2010,The King's Speech,Tom Hooper
0,2010,The Social Network,David Fincher
0,2010,True Grit,Joel Coen and Ethan Coen


In [17]:
# Save the dataframe into a csv file
df_directors.to_csv('Oscar_directors.csv')

### 3.2.2 Animated Film and Best Picture records

In [26]:
# Unlike directing, the number of nominations on Animated film and Best Picture are not always the same
# In general, there are about 8 to 9 best picture nominations, we will assume 9 to max it. 
# Index would include extra elements if the nominations are less than 9
# Extra elements need to not included in the files
pictures = []
for index in pic:
    pictures.append(list_1[index:index+21])

In [27]:
pic_best = []

for i in range(len(pictures)):
    a = [pictures[i][1], pictures[i][3], 
                             pictures[i][5], pictures[i][7], 
         pictures[i][9], pictures[i][11], pictures[i][13], pictures[i][15], 
         pictures[i][17], pictures[i][19]]

    pic_best.append(a)

# Let's experiment the last two and see if we got extra information in
for i in range(len(pic_best)):
    print(pic_best[i][-2:])

['True Grit', "Winter's Bone"]
['War Horse', 'SHORT FILM (Animated)']
['Zero Dark Thirty', 'PRODUCTION DESIGN']
['The Wolf of Wall Street', 'PRODUCTION DESIGN']
['PRODUCTION DESIGN', 'Production Design: Adam Stockhausen; Set Decoration: Anna Pinnock']
['PRODUCTION DESIGN', 'Production Design: Adam Stockhausen; Set Decoration: Rena DeAngelo and Bernhard Henrich']
['Moonlight', 'PRODUCTION DESIGN']
['Three Billboards outside Ebbing, Missouri', 'PRODUCTION DESIGN']
['PRODUCTION DESIGN', 'Production Design: Hannah Beachler; Set Decoration: Jay Hart']
['Parasite', 'PRODUCTION DESIGN']


In [28]:
# We did included extra information
# The ifrst list is fine but other ones need some work
# 6 of them only need to exclude the last item and 3 need to exlude last two 

for i in [1, 2,3,6,7,9]:
    pic_best[i].pop()
for i in [4,5,8]:
    del pic_best[i][-2:]

In [29]:
# Let's find out how many films are nominated each year
for i in range(len(pic_best)):
    print(len(pic_best[i]))

10
9
9
9
8
8
9
9
8
9


In [30]:
# Now we can make the dataframe
y = pd.Series(range(2010,2020))
years = y.repeat([10,9,9,9,8,8,9,9,8,9])
movies =[]
for element in pic_best:
    movies.extend(element)
df_bestpicture = pd.DataFrame({'Year':years,'Movie':movies})
df_bestpicture.head()

,Year,Movie
0,2010,Black Swan
0,2010,The Fighter
0,2010,Inception
0,2010,The Kids Are All Right
0,2010,The King's Speech


In [31]:
df_bestpicture.to_csv('best_picture.csv')

In [32]:
# Doing the same for Animated films
animated = []
for index in ani:
    animated.append(list_1[index:index+11])
animated[0]

['ANIMATED FEATURE FILM',
 'How to Train Your Dragon',
 'Chris Sanders and Dean DeBlois',
 'The Illusionist',
 'Sylvain Chomet',
 'Toy Story 3',
 'Lee Unkrich',
 'ART DIRECTION',
 'Alice in Wonderland',
 "Production Design: Robert Stromberg; Set Decoration: Karen O'Hara",
 'Harry Potter and the Deathly Hallows Part 1']

In [33]:
anim = []
for i in range(len(animated)):
    a = [animated[i][1], animated[i][3], 
                             animated[i][5], animated[i][7], 
        animated[i][9]]
    anim.append(a)
for i in range(len(anim)):
    print(anim[i][-2:])

['ART DIRECTION', "Production Design: Robert Stromberg; Set Decoration: Karen O'Hara"]
['Puss in Boots', 'Rango']
['The Pirates! Band of Misfits', 'Wreck-It Ralph']
['Frozen', 'The Wind Rises']
['Song of the Sea', 'The Tale of the Princess Kaguya']
['Shaun the Sheep Movie', 'When Marnie Was There']
['The Red Turtle', 'Zootopia']
['Ferdinand', 'Loving Vincent']
['Ralph Breaks the Internet', 'Spider-Man: Into the Spider-Verse']
['Missing Link', 'Toy Story 4']


In [34]:
# Only need to clean the first one
for i in [0]:
    del anim[0][-2:]

In [35]:
for i in range(len(anim)):
    print(len(anim[i]))

3
5
5
5
5
5
5
5
5
5


In [36]:
y = pd.Series(range(2010,2020))
years = y.repeat([3,5,5,5,5,5,5,5,5,5])
animations =[]
for element in anim:
    animations.extend(element)
df_animations = pd.DataFrame({'Year':years,'Movie':animations})
df_animations.head()

,Year,Movie
0,2010,How to Train Your Dragon
0,2010,The Illusionist
0,2010,Toy Story 3
1,2011,A Cat in Paris
1,2011,Chico & Rita


In [37]:
df_animations.to_csv('Animated_feature_film.csv')